In [1]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
import pandas as pd
from sklearn.model_selection import train_test_split
import Utilidades as ut
import Modelos as md
import numpy as np
from tensorflow.keras.layers import Activation

Using TensorFlow backend.


In [2]:
##########################################################

# Redefino función de activación 'relu'

def my_relu(x):
    return tf.keras.backend.relu(x, max_value=1.0)

tf.keras.utils.get_custom_objects().update({'my_relu': Activation(my_relu)})

# Redefino función de activación 'swish'

class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    return (tf.keras.backend.sigmoid(x) * x)

tf.keras.utils.get_custom_objects().update({'swish': Swish(swish)})

##########################################################

In [3]:
# Cargamos el dataframe de training
df_train = pd.read_pickle("Archivos/Neuronales_entrenamiento.pkl")
df_test = pd.read_pickle("Archivos/Neuronales_validacion.pkl")

if ('Opportunity_ID' in df_test):
    df_test = df_test.drop(columns=['Opportunity_ID'])

x_train, y_train = ut.split_labels(df_train)
x_test, y_test = ut.split_labels(df_test)

#Convertimos las fechas a numeros (cantidad de dias transcurridos) y luego las normalizamos
x_train, x_test = ut.conversion_fechas(x_train, x_test)
x_train, x_test = ut.codificar_categoricas(x_train, y_train, x_test, modo='catboost')
x_train, x_test = ut.normalizacion_numericas(x_train, x_test, modo='normalizacion')
x_train.shape

x_train_vector = ut.df_a_vector(x_train)
y_train_vector = ut.df_a_vector(y_train)
x_test_vector = ut.df_a_vector(x_test)
y_test_vector = ut.df_a_vector(y_test)

In [4]:
# Definimos el modelo
def model_builder(hp):
    model = keras.Sequential()
    
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    for i in range(hp.Int('layers', min_value = 1, max_value = 5, step = 1)):
        model.add(keras.layers.Dense(units = hp.Int('units', min_value = 16, max_value = 256, step = 16), 
                                     activation = hp.Choice('activation_func', ['relu', 'swish', 'tanh'])))

    model.add(keras.layers.Dense(units = 1,
                                 activation = hp.Choice('activation_func_2', ['relu', 'swish', 'tanh'])))

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = 'binary_crossentropy', 
                metrics = ['accuracy'])

    return model

In [5]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Creamos instancia del tuner
# Keras Tuner tiene cuatro tuner disponibles: RandomSearch , Hyperband , BayesianOptimization y Sklearn
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 50,
#                      max_trials = 5,
#                      factor = 3,
                     directory = 'Archivo',
                     project_name = 'Keras_Tuning')

tuner.search(x_train_vector,
            y_train_vector,
            validation_data=(x_test_vector, y_test_vector),
            epochs = 50,
            verbose=1,
            callbacks = [tf.keras.callbacks.EarlyStopping(patience=1)])
    
models = tuner.get_best_models(num_models=2)
             
tuner.results_summary()

INFO:tensorflow:Reloading Oracle from existing project my_dir\intro_to_kt\oracle.json
INFO:tensorflow:Reloading Tuner from my_dir\intro_to_kt\tuner0.json
Comienza tuneo
Train on 12140 samples, validate on 4515 samples
Epoch 1/2
12140/12140 [==============================] - ETA: 3:56 - loss: 7.2588 - accuracy: 0.40 - ETA: 10s - loss: 1.1365 - accuracy: 0.6903 - ETA: 5s - loss: 0.7370 - accuracy: 0.827 - ETA: 3s - loss: 0.5798 - accuracy: 0.87 - ETA: 2s - loss: 0.4965 - accuracy: 0.90 - ETA: 2s - loss: 0.4812 - accuracy: 0.91 - ETA: 1s - loss: 0.4878 - accuracy: 0.91 - ETA: 1s - loss: 0.4785 - accuracy: 0.92 - ETA: 1s - loss: 0.4548 - accuracy: 0.92 - ETA: 0s - loss: 0.4553 - accuracy: 0.92 - ETA: 0s - loss: 0.4313 - accuracy: 0.93 - ETA: 0s - loss: 0.4146 - accuracy: 0.93 - ETA: 0s - loss: 0.4170 - accuracy: 0.93 - ETA: 0s - loss: 0.4122 - accuracy: 0.94 - ETA: 0s - loss: 0.4092 - accuracy: 0.94 - ETA: 0s - loss: 0.4038 - accuracy: 0.94 - ETA: 0s - loss: 0.3939 - accuracy: 0.94 - 2s 15

Train on 12140 samples, validate on 4515 samples
Epoch 1/2
12140/12140 [==============================] - ETA: 2:07 - loss: 5.8811 - accuracy: 0.37 - ETA: 5s - loss: 2.6845 - accuracy: 0.6363 - ETA: 3s - loss: 2.3002 - accuracy: 0.70 - ETA: 2s - loss: 2.0689 - accuracy: 0.73 - ETA: 1s - loss: 1.9639 - accuracy: 0.75 - ETA: 1s - loss: 1.7521 - accuracy: 0.77 - ETA: 1s - loss: 1.5925 - accuracy: 0.79 - ETA: 0s - loss: 1.4787 - accuracy: 0.80 - ETA: 0s - loss: 1.3319 - accuracy: 0.82 - ETA: 0s - loss: 1.2482 - accuracy: 0.82 - ETA: 0s - loss: 1.1604 - accuracy: 0.83 - ETA: 0s - loss: 1.0896 - accuracy: 0.84 - ETA: 0s - loss: 1.0317 - accuracy: 0.85 - ETA: 0s - loss: 0.9725 - accuracy: 0.86 - ETA: 0s - loss: 0.9199 - accuracy: 0.87 - ETA: 0s - loss: 0.8803 - accuracy: 0.87 - 1s 118us/sample - loss: 0.8575 - accuracy: 0.8807 - val_loss: 0.4933 - val_accuracy: 0.8837
Epoch 2/2
12140/12140 [==============================] - ETA: 1s - loss: 0.1327 - accuracy: 0.93 - ETA: 0s - loss: 0.2412 - ac

Train on 12140 samples, validate on 4515 samples
Epoch 1/2
12140/12140 [==============================] - ETA: 1:48 - loss: 0.8707 - accuracy: 0.46 - ETA: 3s - loss: 0.6437 - accuracy: 0.6677 - ETA: 2s - loss: 0.5023 - accuracy: 0.74 - ETA: 1s - loss: 0.4377 - accuracy: 0.78 - ETA: 1s - loss: 0.4121 - accuracy: 0.81 - ETA: 0s - loss: 0.3730 - accuracy: 0.84 - ETA: 0s - loss: 0.3504 - accuracy: 0.85 - ETA: 0s - loss: 0.3317 - accuracy: 0.87 - ETA: 0s - loss: 0.3148 - accuracy: 0.87 - ETA: 0s - loss: 0.2975 - accuracy: 0.88 - ETA: 0s - loss: 0.2857 - accuracy: 0.89 - ETA: 0s - loss: 0.2754 - accuracy: 0.89 - ETA: 0s - loss: 0.2671 - accuracy: 0.90 - ETA: 0s - loss: 0.2573 - accuracy: 0.90 - 1s 103us/sample - loss: 0.2541 - accuracy: 0.9107 - val_loss: 0.3117 - val_accuracy: 0.9192
Epoch 2/2
12140/12140 [==============================] - ETA: 1s - loss: 0.0261 - accuracy: 1.00 - ETA: 0s - loss: 0.1498 - accuracy: 0.96 - ETA: 0s - loss: 0.1518 - accuracy: 0.96 - ETA: 0s - loss: 0.1361 - ac

Train on 12140 samples, validate on 4515 samples
Epoch 1/2
12140/12140 [==============================] - ETA: 2:15 - loss: 7.3219 - accuracy: 0.25 - ETA: 5s - loss: 2.3569 - accuracy: 0.5498 - ETA: 2s - loss: 1.4367 - accuracy: 0.68 - ETA: 1s - loss: 1.0949 - accuracy: 0.75 - ETA: 1s - loss: 0.8535 - accuracy: 0.80 - ETA: 1s - loss: 0.7238 - accuracy: 0.83 - ETA: 0s - loss: 0.6462 - accuracy: 0.85 - ETA: 0s - loss: 0.5728 - accuracy: 0.87 - ETA: 0s - loss: 0.5178 - accuracy: 0.88 - ETA: 0s - loss: 0.4872 - accuracy: 0.89 - ETA: 0s - loss: 0.4445 - accuracy: 0.90 - ETA: 0s - loss: 0.4252 - accuracy: 0.90 - ETA: 0s - loss: 0.4032 - accuracy: 0.91 - ETA: 0s - loss: 0.3850 - accuracy: 0.91 - ETA: 0s - loss: 0.3696 - accuracy: 0.92 - 1s 117us/sample - loss: 0.3589 - accuracy: 0.9250 - val_loss: 0.4733 - val_accuracy: 0.9216
Epoch 2/2
12140/12140 [==============================] - ETA: 1s - loss: 0.5144 - accuracy: 0.93 - ETA: 0s - loss: 0.2412 - accuracy: 0.97 - ETA: 0s - loss: 0.2192 - ac

Train on 12140 samples, validate on 4515 samples
Epoch 1/2
12140/12140 [==============================] - ETA: 2:07 - loss: 1.1550 - accuracy: 0.40 - ETA: 5s - loss: 0.6353 - accuracy: 0.6680 - ETA: 2s - loss: 0.4918 - accuracy: 0.77 - ETA: 2s - loss: 0.4066 - accuracy: 0.82 - ETA: 1s - loss: 0.3440 - accuracy: 0.85 - ETA: 1s - loss: 0.2992 - accuracy: 0.87 - ETA: 0s - loss: 0.2664 - accuracy: 0.89 - ETA: 0s - loss: 0.2470 - accuracy: 0.90 - ETA: 0s - loss: 0.2288 - accuracy: 0.91 - ETA: 0s - loss: 0.2162 - accuracy: 0.91 - ETA: 0s - loss: 0.2064 - accuracy: 0.92 - ETA: 0s - loss: 0.1959 - accuracy: 0.92 - ETA: 0s - loss: 0.1876 - accuracy: 0.93 - ETA: 0s - loss: 0.1805 - accuracy: 0.93 - ETA: 0s - loss: 0.1755 - accuracy: 0.93 - ETA: 0s - loss: 0.1686 - accuracy: 0.94 - 1s 116us/sample - loss: 0.1696 - accuracy: 0.9428 - val_loss: 0.2562 - val_accuracy: 0.9411
Epoch 2/2
12140/12140 [==============================] - ETA: 1s - loss: 0.1132 - accuracy: 0.96 - ETA: 0s - loss: 0.0836 - ac

...

Train on 12140 samples, validate on 4515 samples
Epoch 1/50
12140/12140 [==============================] - ETA: 2:07 - loss: 3.5371 - accuracy: 0.62 - ETA: 5s - loss: 1.6665 - accuracy: 0.7188 - ETA: 3s - loss: 1.3364 - accuracy: 0.78 - ETA: 2s - loss: 1.1771 - accuracy: 0.81 - ETA: 1s - loss: 1.0120 - accuracy: 0.83 - ETA: 1s - loss: 0.9641 - accuracy: 0.84 - ETA: 1s - loss: 0.8993 - accuracy: 0.85 - ETA: 0s - loss: 0.8454 - accuracy: 0.86 - ETA: 0s - loss: 0.7863 - accuracy: 0.87 - ETA: 0s - loss: 0.7503 - accuracy: 0.88 - ETA: 0s - loss: 0.7181 - accuracy: 0.88 - ETA: 0s - loss: 0.6865 - accuracy: 0.89 - ETA: 0s - loss: 0.6574 - accuracy: 0.89 - ETA: 0s - loss: 0.6353 - accuracy: 0.89 - ETA: 0s - loss: 0.6095 - accuracy: 0.90 - ETA: 0s - loss: 0.5911 - accuracy: 0.90 - ETA: 0s - loss: 0.5694 - accuracy: 0.90 - 2s 131us/sample - loss: 0.5600 - accuracy: 0.9085 - val_loss: 0.6531 - val_accuracy: 0.7962
Epoch 2/50
12140/12140 [==============================] - ETA: 1s - loss: 0.0438 - 

Train on 12140 samples, validate on 4515 samples
Epoch 1/50
12140/12140 [==============================] - ETA: 2:05 - loss: 2.8067 - accuracy: 0.59 - ETA: 6s - loss: 2.0137 - accuracy: 0.8551 - ETA: 3s - loss: 1.8800 - accuracy: 0.86 - ETA: 2s - loss: 1.8682 - accuracy: 0.87 - ETA: 2s - loss: 1.8434 - accuracy: 0.87 - ETA: 1s - loss: 1.7093 - accuracy: 0.88 - ETA: 1s - loss: 1.8398 - accuracy: 0.87 - ETA: 1s - loss: 1.6953 - accuracy: 0.88 - ETA: 0s - loss: 1.7120 - accuracy: 0.88 - ETA: 0s - loss: 1.7220 - accuracy: 0.88 - ETA: 0s - loss: 1.7200 - accuracy: 0.88 - ETA: 0s - loss: 1.7162 - accuracy: 0.88 - ETA: 0s - loss: 1.6978 - accuracy: 0.88 - ETA: 0s - loss: 1.7089 - accuracy: 0.88 - ETA: 0s - loss: 1.6907 - accuracy: 0.88 - ETA: 0s - loss: 1.6848 - accuracy: 0.88 - ETA: 0s - loss: 1.6891 - accuracy: 0.88 - ETA: 0s - loss: 1.6788 - accuracy: 0.88 - 2s 124us/sample - loss: 1.6784 - accuracy: 0.8896 - val_loss: 2.4712 - val_accuracy: 0.8394
Epoch 2/50
12140/12140 [=================

Train on 12140 samples, validate on 4515 samples
Epoch 1/50
12140/12140 [==============================] - ETA: 1:45 - loss: 7.7388 - accuracy: 0.50 - ETA: 3s - loss: 8.3509 - accuracy: 0.4587 - ETA: 2s - loss: 8.4199 - accuracy: 0.45 - ETA: 1s - loss: 8.3086 - accuracy: 0.46 - ETA: 1s - loss: 8.2596 - accuracy: 0.46 - ETA: 0s - loss: 8.3522 - accuracy: 0.45 - ETA: 0s - loss: 8.3643 - accuracy: 0.45 - ETA: 0s - loss: 8.3838 - accuracy: 0.45 - ETA: 0s - loss: 8.3298 - accuracy: 0.46 - ETA: 0s - loss: 8.3200 - accuracy: 0.46 - ETA: 0s - loss: 8.3494 - accuracy: 0.45 - ETA: 0s - loss: 8.3460 - accuracy: 0.45 - ETA: 0s - loss: 8.3539 - accuracy: 0.45 - 1s 98us/sample - loss: 8.3631 - accuracy: 0.4578 - val_loss: 9.3370 - val_accuracy: 0.3947
Epoch 2/50
12140/12140 [==============================] - ETA: 1s - loss: 6.2664 - accuracy: 0.59 - ETA: 0s - loss: 8.4355 - accuracy: 0.45 - ETA: 0s - loss: 8.2989 - accuracy: 0.46 - ETA: 0s - loss: 8.3659 - accuracy: 0.45 - ETA: 0s - loss: 8.3833 - a

Train on 12140 samples, validate on 4515 samples
Epoch 1/50
12140/12140 [==============================] - ETA: 2:16 - loss: 6.7506 - accuracy: 0.56 - ETA: 6s - loss: 8.3754 - accuracy: 0.4570 - ETA: 3s - loss: 8.6372 - accuracy: 0.44 - ETA: 2s - loss: 8.5007 - accuracy: 0.44 - ETA: 1s - loss: 8.4060 - accuracy: 0.45 - ETA: 1s - loss: 8.4781 - accuracy: 0.45 - ETA: 1s - loss: 8.3908 - accuracy: 0.45 - ETA: 0s - loss: 8.3679 - accuracy: 0.45 - ETA: 0s - loss: 8.3612 - accuracy: 0.45 - ETA: 0s - loss: 8.3982 - accuracy: 0.45 - ETA: 0s - loss: 8.3667 - accuracy: 0.45 - ETA: 0s - loss: 8.3840 - accuracy: 0.45 - ETA: 0s - loss: 8.3883 - accuracy: 0.45 - ETA: 0s - loss: 8.3755 - accuracy: 0.45 - ETA: 0s - loss: 8.3537 - accuracy: 0.45 - ETA: 0s - loss: 8.3417 - accuracy: 0.45 - 2s 124us/sample - loss: 8.3630 - accuracy: 0.4578 - val_loss: 9.3370 - val_accuracy: 0.3947
Epoch 2/50
12140/12140 [==============================] - ETA: 1s - loss: 10.1226 - accuracy: 0.343 - ETA: 0s - loss: 8.3552 

INFO:tensorflow:Oracle triggered exit
Fin tuneo


In [12]:
tuner.results_summary()